In [1]:
from helpers.idf_helpers import *
from helpers.plot_helpers import *
from studies import run_study

import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, SelectMultiple
from eppy.modeleditor import IDF
from IPython.display import display

import os

In [2]:
outputs = pd.read_excel('Output Variables.xlsx')

In [3]:
temp = outputs[outputs['Units']=='C']

In [4]:
@interact
def get_output(output = temp['Output Name']):
    return temp[temp['Output Name']==output]

interactive(children=(Dropdown(description='output', options=('Site Outdoor Air Drybulb Temperature', 'Site Ou…

In [5]:
var_list = ['Zone Mean Radiant Temperature',
            'Zone Mean Air Temperature',
            'Zone Operative Temperature',
            'Zone Mean Air Dewpoint Temperature',
            'Zone Outdoor Air Wetbulb Temperature', 
            'Zone Air Temperature',
            'Zone Thermostat Air Temperature',
            'Zone Thermostat Heating Setpoint Temperature',
            'Zone Thermostat Cooling Setpoint Temperature',
              'Zone Adaptive Comfort Operative Temperature Set Point',
           'Zone Heat Index']

In [6]:
idffile = '/home/paula/Documentos/Doctorado/Desarrollo/eppyProject/input/simple_room_1_window_1_door_unitary_System_sch_small_office_Activity.idf'
iddfile = '/usr/local/EnergyPlus-9-4-0/Energy+.idd'
epwfile = '/home/paula/Documentos/Doctorado/Desarrollo/eppyProject/input/Wheather file/FRA_Paris.Orly.071490_IWEC.epw'
idf2 = initialization(idffile, iddfile, epwfile)

In [10]:
@interact
def show_all_classes(classes = idf2.idfobjects):
    return epobject_to_df(classes)

interactive(children=(Dropdown(description='classes', options={'VERSION': [
Version,
    9.4;                 …

In [7]:
@interact
def show_not_null_classes(classes = notnullobj(idf2)):
    return epobject_to_df(idf2.idfobjects[classes])

interactive(children=(Dropdown(description='classes', options=('VERSION', 'SIMULATIONCONTROL', 'BUILDING', 'TI…

## Correcciones en mi idf

In [116]:
#Building from 'SF+CZ5AWH+USA_IL_Chicago-OHare.Intl.AP.725300+hp+crawlspace+IECC_2018'idf2.idfobjects['Building'][0].North_Axis = 0
idf2.idfobjects['Building'][0].Terrain = 'Suburbs'
idf2.idfobjects['Building'][0].Solar_Distribution = 'FullExterior'
idf2.idfobjects['Building'][0].Loads_Convergence_Tolerance_Value = 0.04
idf2.idfobjects['Building'][0].Temperature_Convergence_Tolerance_Value = 0.4
idf2.idfobjects['Building'][0].Maximum_Number_of_Warmup_Days = 25
idf2.idfobjects['Building'][0].Minimum_Number_of_Warmup_Days = 26
idf2.idfobjects['SimulationControl'][0].Run_Simulation_for_Sizing_Periods = 'Yes'
#WinterDesignDay = idf2.idfobjects['SizingPeriod:DesignDay'][0]
#SummerDesignDay = idf2.idfobjects['SizingPeriod:DesignDay'][15]
idf2.idfobjects['HVACTemplate:Thermostat'].Heating_Setpoint_Schedule_Name = ''
# tengo que quitar la temperatura constante en el que se supone que tiene schedule y poner el schedule en temperatura
#borrar el objeto 'SizingPeriod:DesignDay' y añadirle solo estos dos

In [117]:
# save
idf2.save()

In [108]:
idf = idf2
objs_to_delete=[]
scriptname='variable_study'
vars_name=var_list
objs_to_delete = ['PEOPLE', 'LIGHTS', 'ELECTRICEQUIPMENT']

## tengo que hacer cambios para que esta función también funcione desde jupyter notebook

for obj in objs_to_delete:
    for o in idf.idfobjects[obj]:
        idf.removeidfobject(o)
# TODO hacer esto general
# hacer un remove de las variables que ya estaban

for n in range(len(vars_name)):
    idf.newidfobject('OUTPUT:VARIABLE')
    idf.idfobjects['OUTPUT:VARIABLE'][n].Variable_Name = vars_name[n]
# if idf.idfobjects['HVACTemplate:Thermostat']:
#     schedule_name = idf.idfobjects['HVACTemplate:Thermostat'][0].Heating_Setpoint_Schedule_Name
#     idf.newidfobject('OUTPUT:VARIABLE')
#     idf.idfobjects['OUTPUT:VARIABLE'][-1].Key_Value = schedule_name
#     idf.idfobjects['OUTPUT:VARIABLE'][-1].Variable_Name = 'Schedule Value'

# newdir = os.path.basename(__file__).replace('.py','')


foldername = scriptname.replace('.py', '')

try:
    os.mkdir(foldername)
except FileExistsError:
    print("Directory already exists")
abs_newdir = '/home/paula/Documentos/Doctorado/Desarrollo/eppyProject/variable_study'
#idf.saveas(abs_newdir + '/idffile.idf')
idf.run(output_directory=abs_newdir)

Directory already exists

/usr/local/EnergyPlus-9-4-0/energyplus --expandobjects --idd /usr/local/EnergyPlus-9-4-0/Energy+.idd --output-directory /home/paula/Documentos/Doctorado/Desarrollo/eppyProject/variable_study --weather /home/paula/Documentos/Doctorado/Desarrollo/eppyProject/input/Wheather file/FRA_Paris.Orly.071490_IWEC.epw /home/paula/Documentos/Doctorado/Desarrollo/eppyProject/in.idf



In [78]:
%%bash
cd variable_study/
./ReadVarsESO

 ReadVarsESO program starting.
 ReadVars Run Time=00hr 00min  0.18sec
 ReadVarsESO program completed successfully.


In [80]:
data = pd.read_csv('variable_study/eplusout.csv')

In [82]:
data

9192

In [79]:
data = eso_to_ts('variable_study/eplusout.csv')

Exception: dates are duplicated. Impossible to convert to Time Series data frame

In [ ]:
winterperiod = ['2000-01-08','2000-01-10']
summerperiod= ['2000-08-08','2000-08-10']

In [ ]:
datawinter = data[winterperiod[0]:winterperiod[1]]

In [ ]:
datawinter.plot(title = 'invierno',figsize = (12,5),grid =True).legend(loc = 'upper left',bbox_to_anchor=(1.05, 1))

In [ ]:
winter[[]]